In [1]:
!pip3 install 'betterproto[compiler]' websockets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached black-24.10.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (79 kB)
  Using cached protobuf-5.28.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
Using cached black-24.10.0-cp312-cp312-macosx_11_0_arm64.whl (1.4 MB)
Using cached protobuf-5.28.3-cp38-abi3-macosx_10_9_universal2.whl (414 kB)
  Created wheel for betterproto: filename=betterproto-1.2.5-py3-none-any.whl size=21998 sha256=bf8030ab9948cd6c00469270be505d9af53219792da20958d14dc0a18bdeb277
  Stored in directory: /Users/mattihle/Library/Caches/pip/wheels/3f/00/7f/f11c3d3cfd50d910f0edb985199fa3a6a9755bc753c231f

In [ ]:
# Please install:
# betterproto[compiler]
# websockets

from time import sleep
import logging
import uuid
from dataclasses import dataclass, field
from typing import Optional

import betterproto
from typing_extensions import Dict, List
from websockets.frames import CloseCode
from websockets.sync.client import ClientConnection, connect
from dataclasses import dataclass
from typing import List

import betterproto


class Side(betterproto.Enum):
    UNKNOWN = 0
    BID = 1
    OFFER = 2


@dataclass
class Portfolio(betterproto.Message):
    total_balance: float = betterproto.double_field(1)
    available_balance: float = betterproto.double_field(2)
    market_exposures: List["PortfolioMarketExposure"] = betterproto.message_field(3)


@dataclass
class PortfolioMarketExposure(betterproto.Message):
    market_id: int = betterproto.int64_field(1)
    position: float = betterproto.double_field(2)
    total_bid_size: float = betterproto.double_field(3)
    total_offer_size: float = betterproto.double_field(4)
    total_bid_value: float = betterproto.double_field(5)
    total_offer_value: float = betterproto.double_field(6)


@dataclass
class Order(betterproto.Message):
    id: int = betterproto.int64_field(1)
    market_id: int = betterproto.int64_field(2)
    owner_id: str = betterproto.string_field(3)
    transaction_id: int = betterproto.int64_field(4)
    price: float = betterproto.double_field(5)
    size: float = betterproto.double_field(6)
    side: "Side" = betterproto.enum_field(7)
    sizes: List["Size"] = betterproto.message_field(8)


@dataclass
class Size(betterproto.Message):
    transaction_id: int = betterproto.int64_field(1)
    size: float = betterproto.double_field(2)


@dataclass
class Trade(betterproto.Message):
    id: int = betterproto.int64_field(1)
    market_id: int = betterproto.int64_field(2)
    transaction_id: int = betterproto.int64_field(3)
    price: float = betterproto.double_field(4)
    size: float = betterproto.double_field(5)
    buyer_id: str = betterproto.string_field(6)
    seller_id: str = betterproto.string_field(7)


@dataclass
class Market(betterproto.Message):
    id: int = betterproto.int64_field(1)
    name: str = betterproto.string_field(2)
    description: str = betterproto.string_field(3)
    owner_id: str = betterproto.string_field(4)
    transaction_id: int = betterproto.int64_field(5)
    min_settlement: float = betterproto.double_field(6)
    max_settlement: float = betterproto.double_field(7)
    open: "MarketOpen" = betterproto.message_field(8, group="status")
    closed: "MarketClosed" = betterproto.message_field(9, group="status")
    orders: List["Order"] = betterproto.message_field(10)
    trades: List["Trade"] = betterproto.message_field(11)
    has_full_history: bool = betterproto.bool_field(12)


@dataclass
class MarketOpen(betterproto.Message):
    pass


@dataclass
class MarketClosed(betterproto.Message):
    settle_price: float = betterproto.double_field(1)


@dataclass
class MarketSettled(betterproto.Message):
    id: int = betterproto.int64_field(1)
    settle_price: float = betterproto.double_field(2)


@dataclass
class OrderCancelled(betterproto.Message):
    id: int = betterproto.int64_field(1)
    market_id: int = betterproto.int64_field(2)


@dataclass
class OrderCreated(betterproto.Message):
    market_id: int = betterproto.int64_field(1)
    user_id: str = betterproto.string_field(2)
    order: "Order" = betterproto.message_field(3, group="_order")
    fills: List["OrderCreatedOrderFill"] = betterproto.message_field(4)
    trades: List["Trade"] = betterproto.message_field(5)


@dataclass
class OrderCreatedOrderFill(betterproto.Message):
    id: int = betterproto.int64_field(1)
    market_id: int = betterproto.int64_field(2)
    owner_id: str = betterproto.string_field(3)
    size_filled: float = betterproto.double_field(4)
    size_remaining: float = betterproto.double_field(5)
    price: float = betterproto.double_field(6)
    side: "Side" = betterproto.enum_field(7)


@dataclass
class Payment(betterproto.Message):
    id: int = betterproto.int64_field(1)
    payer_id: str = betterproto.string_field(2)
    recipient_id: str = betterproto.string_field(3)
    transaction_id: int = betterproto.int64_field(4)
    amount: float = betterproto.double_field(5)
    note: str = betterproto.string_field(6)


@dataclass
class Payments(betterproto.Message):
    payments: List["Payment"] = betterproto.message_field(1)


@dataclass
class RequestFailed(betterproto.Message):
    request_details: "RequestFailedRequestDetails" = betterproto.message_field(1)
    error_details: "RequestFailedErrorDetails" = betterproto.message_field(2)


@dataclass
class RequestFailedRequestDetails(betterproto.Message):
    kind: str = betterproto.string_field(1)


@dataclass
class RequestFailedErrorDetails(betterproto.Message):
    message: str = betterproto.string_field(1)


@dataclass
class Out(betterproto.Message):
    market_id: int = betterproto.int64_field(1)


@dataclass
class User(betterproto.Message):
    id: str = betterproto.string_field(1)
    name: str = betterproto.string_field(2)
    is_bot: bool = betterproto.bool_field(3)


@dataclass
class Users(betterproto.Message):
    users: List["User"] = betterproto.message_field(1)


@dataclass
class Redeem(betterproto.Message):
    fund_id: int = betterproto.int64_field(1)
    amount: float = betterproto.double_field(2)


@dataclass
class Redeemed(betterproto.Message):
    transaction_id: int = betterproto.int64_field(1)
    user_id: str = betterproto.string_field(2)
    fund_id: int = betterproto.int64_field(3)
    amount: float = betterproto.double_field(4)


@dataclass
class ServerMessage(betterproto.Message):
    request_id: str = betterproto.string_field(19)
    portfolio: "Portfolio" = betterproto.message_field(1, group="message")
    market_data: "Market" = betterproto.message_field(2, group="message")
    market_created: "Market" = betterproto.message_field(3, group="message")
    market_settled: "MarketSettled" = betterproto.message_field(4, group="message")
    order_created: "OrderCreated" = betterproto.message_field(5, group="message")
    order_cancelled: "OrderCancelled" = betterproto.message_field(6, group="message")
    payments: "Payments" = betterproto.message_field(7, group="message")
    payment_created: "Payment" = betterproto.message_field(8, group="message")
    out: "Out" = betterproto.message_field(9, group="message")
    authenticated: "Authenticated" = betterproto.message_field(10, group="message")
    request_failed: "RequestFailed" = betterproto.message_field(11, group="message")
    user_created: "User" = betterproto.message_field(12, group="message")
    users: "Users" = betterproto.message_field(13, group="message")
    acting_as: "ActingAs" = betterproto.message_field(14, group="message")
    ownership_received: "Ownership" = betterproto.message_field(15, group="message")
    ownerships: "Ownerships" = betterproto.message_field(16, group="message")
    ownership_given: "OwnershipGiven" = betterproto.message_field(17, group="message")
    redeemed: "Redeemed" = betterproto.message_field(18, group="message")


@dataclass
class Authenticated(betterproto.Message):
    pass


@dataclass
class ActingAs(betterproto.Message):
    user_id: str = betterproto.string_field(1)


@dataclass
class Ownership(betterproto.Message):
    of_bot_id: str = betterproto.string_field(1)


@dataclass
class Ownerships(betterproto.Message):
    ownerships: List["Ownership"] = betterproto.message_field(1)


@dataclass
class OwnershipGiven(betterproto.Message):
    pass


@dataclass
class MakePayment(betterproto.Message):
    recipient_id: str = betterproto.string_field(1)
    amount: float = betterproto.double_field(2)
    note: str = betterproto.string_field(3)


@dataclass
class CreateMarket(betterproto.Message):
    name: str = betterproto.string_field(1)
    description: str = betterproto.string_field(2)
    min_settlement: float = betterproto.double_field(3)
    max_settlement: float = betterproto.double_field(4)


@dataclass
class SettleMarket(betterproto.Message):
    market_id: int = betterproto.int64_field(1)
    settle_price: float = betterproto.double_field(2)


@dataclass
class CreateOrder(betterproto.Message):
    market_id: int = betterproto.int64_field(2)
    price: float = betterproto.double_field(5)
    size: float = betterproto.double_field(6)
    side: "Side" = betterproto.enum_field(7)


@dataclass
class ClientMessage(betterproto.Message):
    request_id: str = betterproto.string_field(13)
    create_market: "CreateMarket" = betterproto.message_field(1, group="message")
    settle_market: "SettleMarket" = betterproto.message_field(2, group="message")
    create_order: "CreateOrder" = betterproto.message_field(3, group="message")
    cancel_order: "CancelOrder" = betterproto.message_field(4, group="message")
    out: "Out" = betterproto.message_field(5, group="message")
    make_payment: "MakePayment" = betterproto.message_field(6, group="message")
    authenticate: "Authenticate" = betterproto.message_field(7, group="message")
    act_as: "ActAs" = betterproto.message_field(8, group="message")
    create_bot: "CreateBot" = betterproto.message_field(9, group="message")
    give_ownership: "GiveOwnership" = betterproto.message_field(10, group="message")
    upgrade_market_data: "UpgradeMarketData" = betterproto.message_field(
        11, group="message"
    )
    redeem: "Redeem" = betterproto.message_field(12, group="message")


@dataclass
class UpgradeMarketData(betterproto.Message):
    market_id: int = betterproto.int64_field(1)


@dataclass
class CancelOrder(betterproto.Message):
    id: int = betterproto.int64_field(1)


@dataclass
class Authenticate(betterproto.Message):
    jwt: str = betterproto.string_field(1)
    id_jwt: str = betterproto.string_field(2)
    act_as: str = betterproto.string_field(3)


@dataclass
class ActAs(betterproto.Message):
    user_id: str = betterproto.string_field(1)


@dataclass
class CreateBot(betterproto.Message):
    name: str = betterproto.string_field(1)


@dataclass
class GiveOwnership(betterproto.Message):
    of_bot_id: str = betterproto.string_field(1)
    to_user_id: str = betterproto.string_field(2)


logger = logging.getLogger(__name__)


class TradingClient:
    """
    Client for interacting with the exchange server.
    """

    _ws: ClientConnection
    _state: "State"

    def __init__(self, api_url: str, jwt: str, act_as: str):
        """
        Connect, Authenticate, then make sure all of the messages holding initial state have been received.
        """
        self._ws = connect(api_url)
        self._state = State()
        self._outstanding_requests = set()
        authenticate = Authenticate(jwt=jwt, act_as=act_as)
        self.send(ClientMessage(authenticate=authenticate))
        while self._state._initializing:
            server_message = self.recv()
            _, message = betterproto.which_one_of(server_message, "message")
            if isinstance(message, RequestFailed):
                raise RuntimeError(
                    f"{message.request_details.kind} request failed during initialization: {message.error_details.message}"
                )

    def state(self) -> "State":
        """
        Return the up-to-date state of the client.
        """
        try:
            while True:
                self.recv(timeout=1e-100)
        except TimeoutError:
            return self._state

    def create_order(
        self,
        market_id: int,
        price: float,
        size: float,
        side: Side,
    ) -> OrderCreated:
        """
        Place an order on the exchange.
        Note that if price and size are passed as float or Decimal they will be quantized.
        """
        price_quantized = round(price, 2)
        if abs(price_quantized - price) > 1e-4:
            logger.warning(f"Price {price} quantized to {price_quantized}")
        size_quantized = round(size, 2)
        if abs(size_quantized - size) > 1e-4:
            logger.warning(f"Size {size} quantized to {size_quantized}")
        msg = ClientMessage(
            create_order=CreateOrder(
                market_id=market_id,
                price=price_quantized,
                size=size_quantized,
                side=side,
            ),
        )
        response = self.request(msg)
        _, message = betterproto.which_one_of(response, "message")
        assert isinstance(message, OrderCreated)
        return message

    def cancel_order(self, order_id: int) -> OrderCancelled:
        """
        Cancel an order on the exchange.
        """
        msg = ClientMessage(
            cancel_order=CancelOrder(
                id=order_id,
            ),
        )
        response = self.request(msg)
        _, message = betterproto.which_one_of(response, "message")
        assert isinstance(message, OrderCancelled)
        return message

    def out(self, market_id: int) -> Out:
        """
        Cancel all orders for a market.
        """
        msg = ClientMessage(
            out=Out(
                market_id=market_id,
            ),
        )
        response = self.request(msg)
        _, message = betterproto.which_one_of(response, "message")
        assert isinstance(message, Out)
        return message

    def redeem(self, fund_id: int, amount: float) -> Redeemed:
        """
        Redeem a position in a market.
        Note that if amount is passed as float or Decimal it will be quantized.
        """
        amount_quantized = round(amount, 2)
        if abs(amount_quantized - amount) > 1e-4:
            logger.warning(f"Amount {amount} quantized to {amount_quantized}")
        msg = ClientMessage(
            redeem=Redeem(
                fund_id=fund_id,
                amount=amount_quantized,
            ),
        )
        response = self.request(msg)
        _, message = betterproto.which_one_of(response, "message")
        assert isinstance(message, Redeemed)
        return message

    def request(
        self, message: ClientMessage
    ) -> ServerMessage:
        """
        Send a message to the server and wait for a response.
        """
        if not message.request_id:
            message.request_id = str(uuid.uuid4())
        self.send(message)
        while True:
            server_message = self.recv()
            if server_message.request_id == message.request_id:
                _, message = betterproto.which_one_of(server_message, "message")
                if isinstance(message, RequestFailed):
                    if message.request_details.kind == "CancelOrder":
                        logger.error(f"CancelOrder failed:")
                    raise RequestFailed(
                        f"{message.request_details.kind} request failed: {message.error_details.message}"
                    )
                return server_message

    def request_many(
        self, messages: List[ClientMessage]
    ) -> List[ServerMessage]:
        """
        Send a list of messages to the server and wait for responses.
        """
        for message in messages:
            if not message.request_id:
                message.request_id = str(uuid.uuid4())
            self.send(message)
        responses = [ServerMessage() for _ in messages]
        while any(not response.request_id for response in responses):
            server_message = self.recv()
            for i, message in enumerate(messages):
                if server_message.request_id == message.request_id:
                    _, response = betterproto.which_one_of(server_message, "message")
                    if isinstance(response, RequestFailed):
                        raise RequestFailed(
                            f"{response.request_details.kind} request failed: {response.error_details.message}"
                        )
                    responses[i] = server_message
        return responses

    def close(self, code: int = CloseCode.NORMAL_CLOSURE, reason: str = ""):
        """
        Close the connection to the server.
        """
        self._ws.close(code, reason)

    def recv(self, timeout: Optional[float] = None) -> ServerMessage:
        """
        Wait for a message from the server and update the state accordingly,
        returning the kind of message and the message.
        """
        message = self._ws.recv(timeout=timeout)
        assert isinstance(message, bytes)
        decoded = ServerMessage().parse(message)
        self._state._update(decoded)
        return decoded

    def send(self, message: ClientMessage):
        """
        Send a message to the server.
        """
        self._ws.send(bytes(message))

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is None:
            self.close()
        else:
            self.close(CloseCode.INTERNAL_ERROR)


@dataclass
class State:
    """
    Aggregated state from the server
    """

    _initializing: bool = True
    acting_as: ActingAs = field(default_factory=ActingAs)
    portfolio: Portfolio = field(default_factory=Portfolio)
    payments: List[Payment] = field(default_factory=list)
    ownerships: List[Ownership] = field(default_factory=list)
    users: List[User] = field(default_factory=list)
    markets: Dict[int, Market] = field(default_factory=dict)

    def _update(self, server_message: ServerMessage):
        kind, message = betterproto.which_one_of(server_message, "message")

        if isinstance(message, ActingAs):
            # ActingAs is always the last message in the initialization sequence
            self.acting_as = message
            self._initializing = False

        elif isinstance(message, Portfolio):
            self.portfolio = message

        elif isinstance(message, Payments):
            self.payments = message.payments

        elif isinstance(message, Payment):
            assert kind == "payment_created"
            if all(payment.id != message.id for payment in self.payments):
                self.payments.append(message)

        elif isinstance(message, Ownerships):
            self.ownerships = message.ownerships

        elif isinstance(message, Ownership):
            assert kind == "ownership_received"
            if all(
                ownership.of_bot_id != message.of_bot_id
                for ownership in self.ownerships
            ):
                self.ownerships.append(message)

        elif isinstance(message, Users):
            self.users = message.users

        elif isinstance(message, User):
            assert kind == "user_created"
            if all(user.id != message.id for user in self.users):
                self.users.append(message)

        elif isinstance(message, Market):
            self.markets[message.id] = message

        elif isinstance(message, MarketSettled):
            self.markets[message.id].closed = MarketClosed(
                settle_price=message.settle_price
            )

        elif isinstance(message, OrderCancelled):
            self.markets[message.market_id].orders = [
                order
                for order in self.markets[message.market_id].orders
                if order.id != message.id
            ]

        elif isinstance(message, OrderCreated):
            orders = self.markets[message.market_id].orders
            if message.order.id:
                orders.append(message.order)
            if message.fills:
                for order in orders:
                    if fill := next(
                        (fill for fill in message.fills if fill.id == order.id),
                        None,
                    ):
                        order.size = fill.size_remaining
                self.markets[message.market_id].orders = [
                    order for order in orders if float(order.size) > 0
                ]
            if message.trades:
                self.markets[message.market_id].trades.extend(message.trades)


class RequestFailed(Exception):
    pass







# BEGIN USER CODE HERE:
from config import ACT_AS, API_URL, JWT
api_url = API_URL
jwt = JWT
act_as = ACT_AS
client = TradingClient(api_url, jwt, act_as)

ids_to_names = {
    3: "high",
    4: "low",
    5: "sum"
}

while True:
    sleep(0.5) # Not necessary to avoid spamming the server. Just avoids spamming your console.
    state = client.state()
    markets = {}
    for id, name in ids_to_names.items():
        markets[name] = state.markets.get(id)
    
    # print(markets.keys()) # ["high", "low", "sum"]
    # print(markets['high']) # a Market object. either ctrl+f in the file or ask Claude for how to use it.

    # I am going to print some useless data as an example
    for name, market in markets.items():
        total_clips_exchanged = 0
        for trade in market.trades:
            total_clips_exchanged += trade.price * trade.size
        print(f"{total_clips_exchanged:.2f} clips have been traded on {name}.")
    print()